In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupShuffleSplit 
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
buf = np.load("../valid_files.npy", allow_pickle=True).item()
health = buf["health"]
mdd = buf["mdd"]

In [3]:
df = pd.read_csv('ml_prepared_data/entropy.csv')
df["name"] = df["name"].apply(lambda x: x + ".npy")
df = df[df["name"].isin(mdd) | df["name"].isin(health)]

In [4]:
group = df["name"].apply(lambda x: x.split("_")[1])
del df["name"]
df.shape
df.drop(["Unnamed: 0"],axis=1, inplace=True)
df.head()
Y = df["target"]
del df["target"]

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df = pd.DataFrame(scaler.fit_transform(df, Y))
df.head()

,0,1,2,3,4,5,6,7,8,9,...,74,75,76,77,78,79,80,81,82,83
0,-1.757439,-0.449040,-0.395395,-1.145255,-1.567240,-0.438547,-0.065701,-0.985237,-1.557300,-0.409866,...,0.435186,-0.075268,-1.740232,-0.956049,-0.906781,-1.395176,-1.524432,-1.497500,-0.685591,-1.077371
1,0.322928,-0.340274,-0.304368,-0.394565,-0.288210,-1.009268,-1.151696,-0.028394,-0.114164,-0.392503,...,-0.971919,0.120444,0.017888,-0.284445,-0.433761,-0.216211,0.885837,-0.661738,-0.966901,0.648667
2,-0.582189,0.990165,0.668932,-0.581612,0.189428,2.062340,1.110294,0.159962,-0.170741,0.907565,...,0.633466,0.422888,-0.557253,-0.244184,-0.222829,-1.126390,-1.027030,-0.367008,0.154499,-0.567364
3,-0.724065,-0.474787,-0.413329,-0.004417,-1.101528,-0.824246,-0.843351,-0.331109,-0.842338,-0.440359,...,-0.798055,-0.047377,-1.137473,-1.546099,-0.462708,-1.055813,-1.499237,-1.513803,-0.655771,-1.505054
4,-1.132133,-0.926981,-0.405230,-0.687862,-0.997167,-0.551009,-0.141865,-0.058130,-1.226105,-0.813016,...,0.310320,0.831552,-0.947033,-1.148632,-0.853170,-0.442099,-1.956036,-1.080784,-1.057116,-0.245618


In [6]:
splitter = GroupShuffleSplit(test_size=.3, n_splits=2, random_state = 7)
split = splitter.split(df, Y, groups=group)
train_inds, test_inds = next(split)

train = df.iloc[train_inds]
y_train = Y.iloc[train_inds]
train_group = group.iloc[train_inds]

test = df.iloc[test_inds]
y_test = Y.iloc[test_inds]
test_group = group.iloc[test_inds]

test.shape

(869, 84)

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

splitter_cv = GroupShuffleSplit(test_size=.2, n_splits=5, random_state = 42)
split_cv = splitter.split(train, y_train, groups=train_group)



In [8]:
from sklearn.linear_model import LogisticRegression
R = LogisticRegression(max_iter = 5000)

grid = {
    'C':np.linspace(0.001, 0.01, 100)
}

gs = GridSearchCV(R, param_grid=grid, cv=split_cv, scoring="accuracy")
gs.fit(train, y_train)

y_gs = gs.predict(test)
print(accuracy_score(y_test, y_gs))
print(gs.best_estimator_)

0.6651323360184119
LogisticRegression(C=0.0057272727272727275, max_iter=5000)


In [33]:
KN_t = KNeighborsClassifier()

grid_kn = {
    'n_neighbors':range(1, 20),
    'weights': ('uniform', 'distance'),
    'p':(1,2)
}

gs_knc = GridSearchCV(KN_t, param_grid=grid_kn, cv=split_cv, scoring="accuracy")
gs_knc.fit(train, y_train)
y_knc_gs = gs_knc.predict(test)
print(accuracy_score(y_test, y_knc_gs))
print(gs_knc.best_estimator_)

/home/polosatik/.local/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/home/polosatik/.local/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accept

0.6248561565017261
KNeighborsClassifier(n_neighbors=14, p=1)


/home/polosatik/.local/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
